[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/chanwkimlab/MarcoPolo/blob/main/notebooks/tutorial.ipynb)

# Setup

**Start the colab kernel with GPU**: Runtime -> Change runtime type -> GPU

## Install dependencies

In [ ]:
!pip install marcopolo-pytorch --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 614 kB 5.1 MB/s 
     |████████████████████████████████| 96 kB 4.6 MB/s 
     |████████████████████████████████| 2.0 MB 76.1 MB/s 
     |████████████████████████████████| 38.1 MB 15.2 MB/s 
     |████████████████████████████████| 11.2 MB 70.9 MB/s 
     |████████████████████████████████| 930 kB 59.1 MB/s 
     |████████████████████████████████| 88 kB 8.5 MB/s 
     |████████████████████████████████| 1.1 MB 66.2 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=b10f4a5c1d8ec711514f006c15e032a1e0011c009ac3afc1c835508c3d694587
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.7-py3-none-any.whl size=54286 sha256=6407072a8cfb47bb9317310b

In [ ]:
!pip install matplotlib==3.1.3

# Run MarcoPolo

## Import packages

In [ ]:
# Import packages
import pickle

import numpy as np
import pandas as pd
import torch
import anndata as ad
import scanpy as sc
import matplotlib.pyplot as plt

import MarcoPolo

assert torch.cuda.is_available(), "Make sure that you started the colab kernel with GPU: Runtime -> Change runtime type -> GPU"

## Read scRNA-seq data

You can use **example data** or **your own data**.

It should be in a AnnData format. `.X` should contain a raw count matrix of shape (# cells, # genes). You can explore example datasets below

### example data
We have prepared two example data: the human embryogenic stem cell (hESC) dataset of Koh et al. and the liver dataset of MacParland et al.    

In [ ]:
!wget https://raw.githubusercontent.com/chanwkimlab/MarcoPolo/main/notebooks/example/hESC.h5ad
!wget https://raw.githubusercontent.com/chanwkimlab/MarcoPolo/main/notebooks/example/HumanLiver.h5ad
    
anndata_path = "hESC.h5ad"

# Read anndata. `anndata_path` should be in a `h5ad` format.
adata = ad.read(anndata_path)

# For fast debugging, only test first 1,000 genes.
adata = adata[:, :1000]

--2022-06-17 07:12:56--  https://raw.githubusercontent.com/chanwkimlab/MarcoPolo/main/notebooks/example/hESC.h5ad
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20842419 (20M) [application/octet-stream]
Saving to: ‘hESC.h5ad.2’

hESC.h5ad.2         100%[===================>]  19.88M  74.1MB/s    in 0.3s    

2022-06-17 07:12:56 (74.1 MB/s) - ‘hESC.h5ad.2’ saved [20842419/20842419]

--2022-06-17 07:12:56--  https://raw.githubusercontent.com/chanwkimlab/MarcoPolo/main/notebooks/example/HumanLiver.h5ad
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response.

### your own data
You can upload your own AnnData single cell file to this session. If you intend to use the example data, please run the following cell and upload your data.

In [ ]:
from google.colab import files
uploaded = files.upload()

for file_name in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(name=file_name, length=len(uploaded[file_name])))
    
anndata_path = file_name

# Read anndata. `anndata_path` should be in a `h5ad` format.
adata = ad.read(anndata_path)

## (1) Run regression

In [ ]:
# (1) Run regression
# Calculate size factor
if "size_factor" not in adata.obs.columns:
    norm_factor = sc.pp.normalize_total(adata, exclude_highly_expressed=True, max_fraction= 0.2, inplace=False)["norm_factor"]
    adata.obs["size_factor"] = norm_factor/norm_factor.mean()
    print("size factor was calculated")
regression_result = MarcoPolo.run_regression(adata=adata, size_factor_key="size_factor",
                         num_threads=1, device="cuda:0")
# If you use a local machine, you can set `num_threads` to higher than 1 (maybe upto 4), which will speed up the regression a lot. For some reason, num_threads>1 does not seem to work well on colab (maybe due to the the limited RAM).

with open(f"{anndata_path}.regression_result.pickle", "wb") as f:
    pickle.dump(regression_result, f)

The numbers of clusters to test: [1, 2]
Y: (446, 1000) X: (446, 1) s: (446,)
(1) Fitting with 1 cluster(s)


Progress:   0%|          | 0/1000 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/MarcoPolo/regression/models.py:42: UserWarning: Specified kernel cache directory could not be created! This disables kernel caching. Specified directory is /root/.cache/torch/kernels. This warning will appear only once per process. (Triggered internally at  ../aten/src/ATen/native/cuda/jit_utils.cpp:860.)
  return (rate.log() * value) - rate - (value + 1).lgamma()
Progress: 100%|██████████| 1000/1000 [05:28<00:00,  3.04it/s]


(2) Fitting with 2 cluster(s)


Progress: 100%|██████████| 1000/1000 [03:36<00:00,  4.62it/s]


## (2) Find markers

In [ ]:
# (2) Find markers
markers_result = MarcoPolo.find_markers(adata=adata, regression_result=regression_result)
with open(f"{anndata_path}.markers_result.pickle", "wb") as f:
    pickle.dump(markers_result, f)

Assign cells to on-cells and off-cells...
Calculating voting score...
Calculating proximity score...
Calculating bimodality score...
Calculating MarcoPolo score...


## (3) Generate report

In [ ]:
# Obtain tSNE coordinates if it does not exist in the adata.
if "X_tsne" not in adata.obsm.keys():
    sc.tl.tsne(adata=adata)

In [ ]:
# (3) Generate report
MarcoPolo.generate_report(adata=adata, size_factor_key="size_factor", 
                          regression_result=regression_result, 
                          gene_scores=markers_result, 
                          output_dir="./",  
                          low_dim_key="X_tsne",
                          cell_color_key="cell_type",
                          gene_info_path="https://ftp.ncbi.nih.gov/gene/DATA/GENE_INFO/Mammalia/Homo_sapiens.gene_info.gz",
                          top_num_html=1000,
                          top_num_image=1000)

Assign cells to on-cells and off-cells...
Annotating genes with the gene info: https://ftp.ncbi.nih.gov/gene/DATA/GENE_INFO/Mammalia/Homo_sapiens.gene_info.gz


100%|██████████| 1000/1000 [00:22<00:00, 45.07it/s, Num. of unmatched genes=13]


13 not matched genes: MT-CO1, MT-CO3, MT-ND4, MT-ATP6, MT-ND3, MT-CYB, MT-ND2, MT-ND1, MT-ND5, MT-ND4L, MT-ND6, AC090498.1, MT-ATP8
Generating table files...
Generating image files...
Drawing figures
size factor corrected


  0%|          | 0/1000 [00:00<?, ?it/s]findfont: Font family ['Arial'] not found. Falling back to DejaVu Sans.
findfont: Font family ['Arial'] not found. Falling back to DejaVu Sans.
100%|██████████| 1000/1000 [12:39<00:00,  1.32it/s]


# Download report

## compress the report folder 

In [ ]:
!tar -zcvf report.tar.gz report

report/
report/assets/
report/assets/scripts.js
report/assets/details_close.png
report/assets/mp.png
report/assets/styles.css
report/assets/mp_white_large_font.png
report/assets/mp_white.png
report/assets/details_open.png
report/index.html
report/voting_result.html
report/plot_image/
report/plot_image/21.png
report/plot_image/282.png
report/plot_image/100.png
report/plot_image/500.png
report/plot_image/221.png
report/plot_image/427.png
report/plot_image/472.png
report/plot_image/564.png
report/plot_image/501.png
report/plot_image/883.png
report/plot_image/293.png
report/plot_image/409.png
report/plot_image/647.png
report/plot_image/69.png
report/plot_image/103.png
report/plot_image/870.png
report/plot_image/190.png
report/plot_image/528.png
report/plot_image/166.png
report/plot_image/786.png
report/plot_image/556.png
report/plot_image/789.png
report/plot_image/916.png
report/plot_image/186.png
report/plot_image/525.png
report/plot_image/235.png
report/plot_image/555.png
report/plot_ima

## trigger download

In [ ]:
from google.colab import files

files.download('report.tar.gz')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>